# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [6]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [42]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [43]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物学家。你擅长回答生物问题。
之所以如此出色，是因为你掌握了大量的生物学知识，并且能够简单易懂地回答各种问题，
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机学家。你擅长回答计算机问题。
之所以如此出色，是因为你掌握了大量的计算机知识，并且能够简单易懂地回答各种问题，
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

chinese_template = """你是一位很棒的汉语言学家。你擅长回答汉语言学问题。
之所以如此出色，是因为你掌握了大量的汉语言知识，并且能够简单易懂地回答各种问题，
你能够对问题的答案进行归纳总结。

这是一个问题：
{input}"""

In [44]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_template,
    }
]

In [45]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [46]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [47]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [48]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [49]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [50]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [51]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [52]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [53]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< OU

In [54]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [55]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射指的是处于热平衡状态的一个理想化物体所发出的电磁辐射。它的特点是它会以连续的光谱形式发出辐射，且辐射的强度和频率有一定的关系，这就是著名的普朗克定律。黑体辐射在物理学中有重要的应用，例如太阳和恒星的辐射就可以近似地看作是黑体辐射。'}


In [56]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少'}
> Finished chain.
{'input': '大于40的第一个质数是多少', 'text': '？\n\n首先，让我们回顾一下质数的定义：质数是只能被1和自身整除的数字，即除了1和自身外没有其他因数的数字。\n\n现在，我们可以从41开始，逐个检查每个数字是否是质数。如果它是质数，那么它就是大于40的第一个质数。如果它不是质数，我们继续检查下一个数字，直到找到第一个质数为止。\n\n这种方法可能会比较耗时，因此我们可以利用一个数学定理来加快我们的搜索过程，即素数定理。该定理指出，在大于2的数中，第n个质数大约是nln(n)。因此，我们可以用这个公式来估算大于40的第一个质数的位置，然后从这个位置开始向上搜索，直到找到第一个质数为止。\n\n另外，我们还可以利用质'}


In [57]:
router_chain.verbose = True

In [58]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极为密集的天体，它的引力非常强大，甚至连光也无法逃离它的吸引力。它的存在是由于恒星在死亡过程中产生的极其强大的引力。当恒星的质量超过一定的临界值时，它会塌缩成一个非常紧密的物体，形成黑洞。由于黑洞具有非常强大的引力，它会吸引附近的物质，甚至连光也无法逃离，因此它被称为“黑洞”。目前科学家们还在探索黑洞的奥秘，它在宇宙中起着重要的作用，也是我们物理学中的一个重要研究课题。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [59]:
print(chain.invoke("人体的器官有哪些?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '人体器官'}
> Finished chain.
{'input': '人体器官', 'text': '中最大的器官是什么？\n\n人体器官中最大的器官是皮肤。它是人体最外层的覆盖物，平均占据人体总重量的16%。皮肤的主要功能有保护身体免受外界伤害、调节体温、排泄代谢产物等。'}


In [60]:
print(chain.invoke("描述RESTful API的原则，并解释如何在Web服务中实现它们"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '描述RESTful API的原则，并解释如何在Web服务中实现它们'}
> Finished chain.
{'input': '描述RESTful API的原则，并解释如何在Web服务中实现它们', 'history': '', 'text': ' RESTful API有几个基本原则，包括客户端-服务器架构、无状态、缓存、统一接口和可伸缩性。在Web服务中实现这些原则需要一些步骤。首先，客户端和服务器之间的交互应该是通过HTTP协议进行的，客户端发送请求，服务器返回响应。其次，服务器应该是无状态的，即不保存客户端的状态信息，每次请求都是独立的。第三，服务器可以使用缓存来提高性能，客户端可以缓存响应，避免重复请求。第四，统一接口是指使用统一的方式来处理请求和响应，例如使用HTTP的GET、POST、PUT和DELETE方法。最后，Web服务应该具有可伸缩性，即能够处理大量的请求并保持稳定。要实现这些原则，开发者可以使用一些工具和框架，例如Express'}


In [61]:
print(chain.invoke("解释一下什么是面向对象编程（OOP）及其主要特征?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '解释一下什么是面向对象编程（OOP）及其主要特征?'}
> Finished chain.
{'input': '解释一下什么是面向对象编程（OOP）及其主要特征?', 'text': '\n\n面向对象编程（Object-Oriented Programming，OOP）是一种编程范式，它将现实世界中的事物抽象成对象，并且通过对象之间的交互来解决问题。\n\nOOP的主要特征包括封装、继承和多态。\n\n1. 封装：封装指的是将数据和方法封装在对象内部，通过对象的接口来访问和操作数据，从而保证数据的安全性和完整性。\n\n2. 继承：继承指的是通过建立父子类关系来实现代码的重用。子类可以继承父类的属性和方法，并且可以通过重写和调用父类的方法来扩展和修改父类的功能。\n\n3. 多态：多态指的是同一种方法在不同的对象上具有不同的行为。通过多态，可以实现代码的灵活性和可扩展性。\n\n除了'}


In [62]:
print(chain.invoke("请解释汉语中的四声，并举例说明每个声调的发音变化?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言: {'input': '请解释汉语中的四声，并举例说明每个声调的发音变化?'}
> Finished chain.
{'input': '请解释汉语中的四声，并举例说明每个声调的发音变化?', 'text': '\n\n汉语中的四声是指平声、上声、去声和入声。每个声调都是通过声调符号来表示的，分别是平声（无声调符号）、上声（´）、去声（`）、入声（ˇ）。每个声调都有不同的发音变化。\n\n1. 平声：平声是指发音时声音平稳，没有明显的上升或下降。例如，“妈妈”、“爸爸”、“书”、“水”等词都是平声。发音时，声调保持平稳不变。\n\n2. 上声：上声是指发音时声调上升，声音由低到高。例如，“看”、“衣服”、“笔”等词都是上声。发音时，声调从低到高，声音由轻到重。\n\n3. 去声：去声是指发音时声调下降，声音由高到低。例如，“东西”、“'}
